### Imports and hyperparameters

In [1]:
import pandas as pd
import numpy as np

import os

from transformers import AutoTokenizer, TFAutoModel

import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef, roc_auc_score

In [2]:
# Hyperparameters
MAX_SEQUENCE_LENGTH = 100
EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 3e-5
DROPOUT = 0.2

# Set a seed to decrease randomness
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

### Load development dataset

In [3]:
valid = pd.read_csv('dev.csv')

### Set up tokenizer for DeBERTa model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Tokenising and preparing inputs

In [5]:
# Encode data with a max length of 100
def bert_encode(hypotheses, premises, tokenizer, max_length=MAX_SEQUENCE_LENGTH):

    x = tokenizer(hypotheses, premises, padding='max_length', truncation=True, max_length=max_length)

    inputs = {
          'input_word_ids':tf.ragged.constant(x['input_ids']).to_tensor(),
          'input_mask': tf.ragged.constant(x['attention_mask']).to_tensor(),
          'input_type_ids': tf.ragged.constant(x['token_type_ids']).to_tensor()}

    return inputs

In [6]:
valid_input = bert_encode(valid.premise.values.tolist(), valid.hypothesis.values.tolist(), tokenizer)

### Define model

In [7]:
os.environ["WANDB_API_KEY"] = "0"

def build_model():
    bert_encoder = TFAutoModel.from_pretrained("microsoft/deberta-v3-base")
    input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name="input_type_ids")

    output = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.GlobalAveragePooling1D()(output)

    output = tf.keras.layers.Dropout(DROPOUT)(output)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(output)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)

    return model

In [8]:
model = build_model()
model.summary()

tf_model.h5:   0%|          | 0.00/736M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaV2Model.

All the layers of TFDebertaV2Model were initialized from the model checkpoint at microsoft/deberta-v3-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2Model for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 100)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 100)]                0         []                            
                                                                                                  
 input_type_ids (InputLayer  [(None, 100)]                0         []                            
 )                                                                                                
                                                                                              

In [9]:
# Load pretrained model
model.load_weights('best.model.weights.h5')

/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


### Compute evaluation metrics

In [10]:
outputs = model.predict(valid_input)

preds = outputs > 0.5
labels = valid.label.values.reshape(-1,1)

print(f"Accuracy: {accuracy_score(labels, preds):.4f}")
print(f"F1 Score: {f1_score(labels, preds):.4f}")
print(f"Precision: {precision_score(labels, preds):.4f}")
print(f"Recall: {recall_score(labels, preds):.4f}")
print(f"MCC: {matthews_corrcoef(labels, preds):.4f}")
print(f"ROC AUC Score: {roc_auc_score(labels, preds):.4f}")

211/211 [==============================] - 60s 247ms/step
Accuracy: 0.9234
F1 Score: 0.9265
Precision: 0.9179
Recall: 0.9353
MCC: 0.8467
ROC AUC Score: 0.9230
